In [1]:
%pylab inline
import pandas as pd
import sklearn.ensemble as sk_ensemble
import sklearn.cross_validation as sk_cv
import xgboost as xgb

Populating the interactive namespace from numpy and matplotlib


In [2]:
def rmsle(actual, predicted):
    error = np.log1p(predicted) - np.log1p(actual)
    return np.sqrt(np.mean(np.square(error)))

In [3]:
train_df = pd.read_csv('../train_set_adjusted.csv')

In [4]:
# print train_df.shape
# print train_df.columns

In [5]:
tube = pd.read_csv('../tube_material_id_imputed_dummies_drop_ns.csv')

In [6]:
# print tube.shape
# print tube.columns

In [7]:
spec = pd.read_csv('../spec_dummies.csv')

In [8]:
# print spec.shape
# print spec.columns

In [9]:
comp = pd.read_csv('../comp_type_dummies.csv')

In [10]:
# print comp.shape
# print comp.columns

In [11]:
comp_type_weight = pd.read_csv('../comp_type_weight.csv')

In [12]:
# print comp_type_weight.shape
# print comp_type_weight.columns

In [13]:
tube_vol = pd.read_csv('../tube_volume.csv')

In [14]:
train = pd.merge(train_df, tube)
train = pd.merge(train, spec)
train = pd.merge(train, comp_type_weight)
train = pd.merge(train, tube_vol)

In [15]:
train.shape

(30213, 264)

In [16]:
train_sub_train_cv, train_sub_test = sk_cv.train_test_split(train, train_size = 0.7, random_state = 0)
train_sub_train, train_sub_cv = sk_cv.train_test_split(train_sub_train_cv, train_size = 0.7, random_state = 0)

In [17]:
print train_sub_train.shape
print train_sub_cv.shape

(14804, 264)
(6345, 264)


In [18]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.1
# params["min_child_weight"] = 5
# params["subsample"] = 1.0
# params["scale_pos_weight"] = 1.0
# params["silent"] = 1
# params["max_depth"] = 7
plst = list(params.items())

In [19]:
X_train = train_sub_train.drop(['tube_assembly_id', 'quote_date', 'cost'], axis=1).values
y_train = train_sub_train.cost
X_cv = train_sub_cv.drop(['tube_assembly_id', 'quote_date', 'cost'], axis=1).values
y_cv = train_sub_cv.cost
X_test = train_sub_test.drop(['tube_assembly_id', 'quote_date', 'cost'], axis=1).values
y_test = train_sub_test.cost

In [20]:
xgtrain = xgb.DMatrix(X_train, label=np.log1p(y_train))
xgcv = xgb.DMatrix(X_cv)
N = [1500,2000,2500]
for i,num_rounds in enumerate(N):
    # num_rounds = 120
    model = xgb.train(plst, xgtrain, num_rounds)
    cv_preds = np.expm1(model.predict(xgcv))
    print N[i],rmsle(y_cv, cv_preds)

1500 0.200247684282
2000 0.197408971549
2500 0.195893214808


In [21]:
test_preds = np.expm1(model.predict(xgb.DMatrix(X_test)))
rmsle(y_test, test_preds)

0.18927403884481925

In [22]:
kaggle_test = pd.read_csv('../test_dummies_adjusted.csv')

In [23]:
kaggle_test_df = pd.merge(kaggle_test, tube)
kaggle_test_df = pd.merge(kaggle_test_df, spec)
kaggle_test_df = pd.merge(kaggle_test_df, comp_type_weight)
kaggle_test_df = pd.merge(kaggle_test_df, tube_vol)

In [24]:
# print kaggle_test_df.id

In [25]:
kaggle_test_vals = kaggle_test_df.drop(['tube_assembly_id', 'quote_date', 'cost', 'id'], axis=1).values

In [26]:
kaggle_preds = np.expm1(model.predict(xgb.DMatrix(kaggle_test_vals)))

In [27]:
print kaggle_preds[:10]

[ 22.57033348  14.05496883   8.27944088   6.19515324   4.17370701
   3.52834892   3.09796882   2.81846285  21.8551693   12.88787937]


In [28]:
print kaggle_preds.shape

(30235,)


In [29]:
submit = pd.DataFrame({'id' : range(1, 30236), 'cost' : kaggle_preds})

In [30]:
submit.tail(10)

,cost,id
30225,13.411424,30226
30226,7.448846,30227
30227,4.965157,30228
30228,15.013838,30229
30229,5.429459,30230
30230,10.219048,30231
30231,5.092091,30232
30232,4.906714,30233
30233,34.920242,30234
30234,39.599998,30235


In [34]:
submit.to_csv('../submit_py_xgboost.csv', index=False, columns=['id', 'cost'])